In [1]:
import pyspark
import pandas as pd
from pyspark.sql import SparkSession, types
from pyspark.sql import functions as F
from datetime import datetime
from collections import namedtuple
import time

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/23 17:05:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_green = spark.read.parquet('../code/data/pq/green/*/*')

In [4]:
'''
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records

FROM
    green_data
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
'''

"\nSELECT \n    date_trunc('hour', lpep_pickup_datetime) AS hour, \n    PULocationID AS zone,\n    SUM(total_amount) AS amount,\n    COUNT(1) AS number_records\n\nFROM\n    green_data\nWHERE\n    lpep_pickup_datetime >= '2020-01-01 00:00:00'\nGROUP BY\n    1, 2\n"

In [5]:
df_green.rdd.take(5)

[Row(VendorID=1, lpep_pickup_datetime=datetime.datetime(2020, 1, 3, 19, 0, 1), lpep_dropoff_datetime=datetime.datetime(2020, 1, 3, 19, 5, 48), store_and_fwd_flag='N', RatecodeID=1, PULocationID=244, DOLocationID=116, passenger_count=1, trip_distance=1.0, fare_amount=6.0, extra=1.0, mta_tax=0.5, tip_amount=1.0, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=8.8, payment_type=1, trip_type=1, congestion_surcharge=0.0),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 29, 19, 55, 1), lpep_dropoff_datetime=datetime.datetime(2020, 1, 29, 19, 59, 12), store_and_fwd_flag='N', RatecodeID=1, PULocationID=166, DOLocationID=24, passenger_count=1, trip_distance=0.82, fare_amount=5.0, extra=1.0, mta_tax=0.5, tip_amount=1.36, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=8.16, payment_type=1, trip_type=1, congestion_surcharge=0.0),
 Row(VendorID=1, lpep_pickup_datetime=datetime.datetime(2020, 1, 2, 10, 20, 42), lpep_dropoff_dateti

In [6]:
df_green.take(5)

[Row(VendorID=1, lpep_pickup_datetime=datetime.datetime(2020, 1, 3, 19, 0, 1), lpep_dropoff_datetime=datetime.datetime(2020, 1, 3, 19, 5, 48), store_and_fwd_flag='N', RatecodeID=1, PULocationID=244, DOLocationID=116, passenger_count=1, trip_distance=1.0, fare_amount=6.0, extra=1.0, mta_tax=0.5, tip_amount=1.0, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=8.8, payment_type=1, trip_type=1, congestion_surcharge=0.0),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 29, 19, 55, 1), lpep_dropoff_datetime=datetime.datetime(2020, 1, 29, 19, 59, 12), store_and_fwd_flag='N', RatecodeID=1, PULocationID=166, DOLocationID=24, passenger_count=1, trip_distance=0.82, fare_amount=5.0, extra=1.0, mta_tax=0.5, tip_amount=1.36, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=8.16, payment_type=1, trip_type=1, congestion_surcharge=0.0),
 Row(VendorID=1, lpep_pickup_datetime=datetime.datetime(2020, 1, 2, 10, 20, 42), lpep_dropoff_dateti

## WHERE vs filter() 

In [7]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [8]:
rdd.filter(lambda x: True).take(1)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 3, 19, 0, 1), PULocationID=244, total_amount=8.8)]

In [9]:
rdd.filter(lambda x: False).take(1)

[]

In [10]:
start_date = datetime(year=2020, month=1, day=1)

def filter_outliers(row):
    return row['lpep_pickup_datetime'] >= start_date

In [11]:
rdd \
    .filter(filter_outliers) \
    .take(2)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 3, 19, 0, 1), PULocationID=244, total_amount=8.8),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 29, 19, 55, 1), PULocationID=166, total_amount=8.16)]

## GROUP BY vs map() 

In [12]:
row = rdd.take(1)[0]
row

Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 3, 19, 0, 1), PULocationID=244, total_amount=8.8)

In [13]:
row['lpep_pickup_datetime'].replace(minute=0, second=0, microsecond=0)

datetime.datetime(2020, 1, 3, 19, 0)

In [14]:
def prepare_for_grouping(row):
    hour = row['lpep_pickup_datetime'].replace(minute=0, second=0, microsecond=0)
    zone = row['PULocationID']
    key = (hour, zone)
    
    amount = row['total_amount']
    count = 1
    value = (amount, count)
    
    return (key, value)

In [17]:
rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .take(10)

[((datetime.datetime(2020, 1, 3, 19, 0), 244), (8.8, 1)),
 ((datetime.datetime(2020, 1, 29, 19, 0), 166), (8.16, 1)),
 ((datetime.datetime(2020, 1, 2, 10, 0), 145), (3.3, 1)),
 ((datetime.datetime(2020, 1, 23, 10, 0), 101), (31.79, 1)),
 ((datetime.datetime(2020, 1, 12, 17, 0), 74), (5.3, 1)),
 ((datetime.datetime(2020, 1, 27, 19, 0), 134), (12.09, 1)),
 ((datetime.datetime(2020, 1, 24, 21, 0), 42), (7.25, 1)),
 ((datetime.datetime(2020, 1, 4, 17, 0), 82), (19.56, 1)),
 ((datetime.datetime(2020, 1, 7, 5, 0), 222), (29.88, 1)),
 ((datetime.datetime(2020, 1, 8, 17, 0), 226), (27.92, 1))]

## Reduce by key

In [18]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)

In [19]:
rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .take(10)

[((datetime.datetime(2020, 1, 2, 10, 0), 145), (119.76, 17)),
 ((datetime.datetime(2020, 1, 12, 17, 0), 74), (1115.0299999999995, 83)),
 ((datetime.datetime(2020, 1, 14, 14, 0), 181), (332.53000000000003, 17)),
 ((datetime.datetime(2020, 1, 3, 9, 0), 177), (45.989999999999995, 2)),
 ((datetime.datetime(2020, 1, 8, 10, 0), 95), (556.0999999999999, 28)),
 ((datetime.datetime(2020, 1, 29, 20, 0), 65), (380.83000000000015, 27)),
 ((datetime.datetime(2020, 1, 24, 16, 0), 223), (334.2800000000001, 27)),
 ((datetime.datetime(2020, 1, 19, 10, 0), 166), (969.2699999999991, 55)),
 ((datetime.datetime(2020, 1, 23, 19, 0), 116), (78.69999999999999, 5)),
 ((datetime.datetime(2020, 1, 24, 11, 0), 82), (277.7800000000001, 18))]

## Unwrap the row 

In [20]:
def unwarp(row):
    return (row[0][0], row[0][1], row[1][0], row[1][1])

In [21]:
rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwarp) \
    .take(10)

[(datetime.datetime(2020, 1, 2, 10, 0), 145, 119.76, 17),
 (datetime.datetime(2020, 1, 12, 17, 0), 74, 1115.0299999999995, 83),
 (datetime.datetime(2020, 1, 14, 14, 0), 181, 332.53000000000003, 17),
 (datetime.datetime(2020, 1, 3, 9, 0), 177, 45.989999999999995, 2),
 (datetime.datetime(2020, 1, 8, 10, 0), 95, 556.0999999999999, 28),
 (datetime.datetime(2020, 1, 29, 20, 0), 65, 380.83000000000015, 27),
 (datetime.datetime(2020, 1, 24, 16, 0), 223, 334.2800000000001, 27),
 (datetime.datetime(2020, 1, 19, 10, 0), 166, 969.2699999999991, 55),
 (datetime.datetime(2020, 1, 23, 19, 0), 116, 78.69999999999999, 5),
 (datetime.datetime(2020, 1, 24, 11, 0), 82, 277.7800000000001, 18)]

In [22]:
rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwarp) \
    .toDF() \
    .show(5)

+-------------------+---+------------------+---+
|                 _1| _2|                _3| _4|
+-------------------+---+------------------+---+
|2020-01-02 10:00:00|145|            119.76| 17|
|2020-01-12 17:00:00| 74|1115.0299999999995| 83|
|2020-01-14 14:00:00|181|332.53000000000003| 17|
|2020-01-03 09:00:00|177|45.989999999999995|  2|
|2020-01-08 10:00:00| 95| 556.0999999999999| 28|
+-------------------+---+------------------+---+
only showing top 5 rows



#### Saving the name of the row 

In [23]:
RevenueRow = namedtuple('revenuerow', ['hour', 'zone', 'revenue', 'count'])

In [24]:
def unwarp_with_name(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1], 
        revenue=row[1][0], 
        count=row[1][1])

In [25]:
# without predefined schema

start = time.time()

df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwarp_with_name) \
    .toDF()

end = time.time()

print(f'{end-start:02f} s')


16.422429 s


In [26]:
df_result.show(5)

+-------------------+----+------------------+-----+
|               hour|zone|           revenue|count|
+-------------------+----+------------------+-----+
|2020-01-02 10:00:00| 145|            119.76|   17|
|2020-01-12 17:00:00|  74|1115.0299999999995|   83|
|2020-01-14 14:00:00| 181|332.53000000000003|   17|
|2020-01-03 09:00:00| 177|45.989999999999995|    2|
|2020-01-08 10:00:00|  95| 556.0999999999999|   28|
+-------------------+----+------------------+-----+
only showing top 5 rows



In [27]:
df_result.schema

StructType([StructField('hour', TimestampType(), True), StructField('zone', LongType(), True), StructField('revenue', DoubleType(), True), StructField('count', LongType(), True)])

In [28]:
result_schema = types.StructType([
            types.StructField('hour', types.TimestampType(), True), 
            types.StructField('zone', types.IntegerType(), True), 
            types.StructField('revenue', types.DoubleType(), True), 
            types.StructField('count', types.IntegerType(), True)])

In [29]:
# with predefined schema

start = time.time()

df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwarp_with_name) \
    .toDF(result_schema)

end = time.time()

print(f'{end-start} s')


0.031581878662109375 s


In [30]:
df_result.show(5)

+-------------------+----+------------------+-----+
|               hour|zone|           revenue|count|
+-------------------+----+------------------+-----+
|2020-01-02 10:00:00| 145|            119.76|   17|
|2020-01-12 17:00:00|  74|1115.0299999999995|   83|
|2020-01-14 14:00:00| 181|332.53000000000003|   17|
|2020-01-03 09:00:00| 177|45.989999999999995|    2|
|2020-01-08 10:00:00|  95| 556.0999999999999|   28|
+-------------------+----+------------------+-----+
only showing top 5 rows



#### Save to parquet 

In [32]:
df_result.write.parquet?

In [33]:
df_result.write.parquet('tmp/green-revenue', mode='overwrite')

## Map Partition 

In [35]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID']

duration_rdd = df_green \
    .select(columns) \
    .rdd

## Using MapPartition 

In [36]:
def apply_model_in_batch(partition):
    cnt = 0
    
    for row in partition:
        cnt += 1
        
    return [cnt]

In [37]:
rdd.mapPartitions(apply_model_in_batch).collect()

[1141148, 438057, 432402, 292910]

## Turn partition into pandas DF 

In [38]:
rows = duration_rdd.take(10)
rows

[Row(VendorID=1, lpep_pickup_datetime=datetime.datetime(2020, 1, 3, 19, 0, 1), PULocationID=244, DOLocationID=116),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 29, 19, 55, 1), PULocationID=166, DOLocationID=24),
 Row(VendorID=1, lpep_pickup_datetime=datetime.datetime(2020, 1, 2, 10, 20, 42), PULocationID=145, DOLocationID=145),
 Row(VendorID=None, lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 10, 24), PULocationID=101, DOLocationID=203),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 12, 17, 30, 50), PULocationID=74, DOLocationID=74),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 27, 19, 33, 9), PULocationID=134, DOLocationID=135),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 24, 21, 25, 12), PULocationID=42, DOLocationID=42),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 4, 17, 6, 40), PULocationID=82, DOLocationID=223),
 Row(VendorID=None, lpep_pickup_datetime=datetime.datetime(2020

In [39]:
pd.DataFrame(rows, columns=columns)

,VendorID,lpep_pickup_datetime,PULocationID,DOLocationID
0,1.0,2020-01-03 19:00:01,244,116
1,2.0,2020-01-29 19:55:01,166,24
2,1.0,2020-01-02 10:20:42,145,145
3,NaN,2020-01-23 10:24:00,101,203
4,2.0,2020-01-12 17:30:50,74,74
5,2.0,2020-01-27 19:33:09,134,135
6,2.0,2020-01-24 21:25:12,42,42
7,2.0,2020-01-04 17:06:40,82,223
8,NaN,2020-01-07 05:46:00,222,80
9,NaN,2020-01-08 17:18:00,226,167


In [40]:
def apply_model_in_batch_pandas(rows):
    
    df = pd.DataFrame(rows, columns=columns)
    
    cnt = len(df)
        
    return [cnt]

In [41]:
rdd.mapPartitions(apply_model_in_batch).collect()

[1141148, 438057, 432402, 292910]

## Machine learning in partition 

#### df.itertuples 

In [42]:
df = pd.DataFrame(rows, columns=columns)

In [43]:
list(df.itertuples())

[Pandas(Index=0, VendorID=1.0, lpep_pickup_datetime=Timestamp('2020-01-03 19:00:01'), PULocationID=244, DOLocationID=116),
 Pandas(Index=1, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-29 19:55:01'), PULocationID=166, DOLocationID=24),
 Pandas(Index=2, VendorID=1.0, lpep_pickup_datetime=Timestamp('2020-01-02 10:20:42'), PULocationID=145, DOLocationID=145),
 Pandas(Index=3, VendorID=nan, lpep_pickup_datetime=Timestamp('2020-01-23 10:24:00'), PULocationID=101, DOLocationID=203),
 Pandas(Index=4, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-12 17:30:50'), PULocationID=74, DOLocationID=74),
 Pandas(Index=5, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-27 19:33:09'), PULocationID=134, DOLocationID=135),
 Pandas(Index=6, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-24 21:25:12'), PULocationID=42, DOLocationID=42),
 Pandas(Index=7, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-04 17:06:40'), PULocationID=82, DOLocationID=223),
 Pandas(Index=8, Vendo

#### yield in Python 

In [44]:
def infinite_seq():
    i = 0
    
    while True:
        yield i
        i += 1
        
        if i > 15:
            break

In [45]:
seq = infinite_seq()

In [46]:
list(seq)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [47]:
#model = ...

def model_predict(df):
#     y_pred = model.predict(df)
    y_pred = df.trip_distance * 5.2
    return y_pred 

In [59]:
columns_ml = ['lpep_pickup_datetime', 'PULocationID', 'total_amount', 'trip_distance']

def apply_ml_model_in_batch(rows):
    
    df = pd.DataFrame(rows, columns=columns_ml)
    predictions = model_predict(df)
    df['predicted_duration'] = predictions
    
    for row in df.itertuples():
        yield row

In [60]:
rdd_ml = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount', 'trip_distance') \
    .rdd

In [61]:
rdd_ml.mapPartitions(apply_ml_model_in_batch).take(10)

[Pandas(Index=0, lpep_pickup_datetime=Timestamp('2020-01-03 19:00:01'), PULocationID=244, total_amount=8.8, trip_distance=1.0, predicted_duration=5.2),
 Pandas(Index=1, lpep_pickup_datetime=Timestamp('2020-01-29 19:55:01'), PULocationID=166, total_amount=8.16, trip_distance=0.82, predicted_duration=4.264),
 Pandas(Index=2, lpep_pickup_datetime=Timestamp('2020-01-02 10:20:42'), PULocationID=145, total_amount=3.3, trip_distance=0.0, predicted_duration=0.0),
 Pandas(Index=3, lpep_pickup_datetime=Timestamp('2020-01-23 10:24:00'), PULocationID=101, total_amount=31.79, trip_distance=4.88, predicted_duration=25.376),
 Pandas(Index=4, lpep_pickup_datetime=Timestamp('2020-01-12 17:30:50'), PULocationID=74, total_amount=5.3, trip_distance=0.34, predicted_duration=1.7680000000000002),
 Pandas(Index=5, lpep_pickup_datetime=Timestamp('2020-01-27 19:33:09'), PULocationID=134, total_amount=12.09, trip_distance=1.59, predicted_duration=8.268),
 Pandas(Index=6, lpep_pickup_datetime=Timestamp('2020-01-2